In [ ]:
import pandas as pd
from shapely.ops import unary_union
import shapely
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import Point
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import re
import glob
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt



def download(des_loc,
            http_page,
            str1,
            str2):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/shapefile_standardization
    @ author's email id:       sh.gharari@gmail.com
    @license:                  MIT

    This function gets name of a http and two str in the name of the link and save them in
    provided destnation
    

    Arguments
    ---------
    des_loc: string, the name of the source file including path and extension
    http_page: string, the name of the corresponding catchment (subbasin) for the unresolved hills
    str1: string, a part of the link name to filter
    str2: string, a second part of the link name to filter
    

    Returns
    -------


    Saves Files
    -------
    downlaod the files from the websites and save them in the correct location
    """

    # first get all the links in the page
    req = Request(http_page)
    html_page = urlopen(req)
    soup = BeautifulSoup(html_page, "lxml")
    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))

    # specify the link to be downloaded
    link_to_download = []
    for link in links:
        # if "hillslope" in link and "clean" in link: # links that have cat_pfaf and Basins in them
        if str1 in link and str2 in link: # links that have cat_pfaf and Basins in them
            link_to_download.append(link)
            print(link)

    # creat urls to download
    urls =[]
    for file_name in link_to_download:
        urls.append(http_page+file_name) # link of the page + file names
        print(http_page+file_name)
    print(urls)

    # loop to download the data
    for url in urls:
        name = url.split('/')[-1] # get the name of the file at the end of the url to download
        r = requests.get(url) # download the URL
        # print the specification of the download 
        print(r.status_code, r.headers['content-type'], r.encoding)
        # if download successful the statuse code is 200 then save the file, else print not downloaded
        if r.status_code == 200:
            print('download was successful for '+url)
            with open(des_loc+name, 'wb') as f:
                f.write(r.content)
        else:
            print('download was not successful for '+url)
    


## Download section and input path
### A list of IDs based on 2 digit pfaf code are provided for download, the path to save the donwload is provided and also the website to download the model 

In [ ]:
# the 2 digit pfaf code for the shapefile to be processed
# list of IDs for downloading the processing
IDs = ['11', '12', '13', '14', '15', '16', '17', '18',
       '21', '22', '23', '24', '25', '26', '27', '28', '29',
       '31', '32', '33', '34', '35', '36',
       '41', '42', '43', '44', '45', '46', '47', '48', '49',
       '51', '52', '53', '54', '55', '56', '57',
       '61', '62', '63', '64', '65', '66', '67',
       '71', '72', '73', '74', '75', '76', '77', '78',
       '81', '82', '83', '84', '85', '86',
       '91']
# location of files online
http_path = 'XXX/for_martyn/' # link to the page that the data exists
# in this folder create subfolders cat, riv, hill, cat_step_1,cat_step_2
path = '/Users/shg096/Desktop/MERIT_Hydro/'

In [ ]:
# STEP- prepare the folder and subfolders for download
# path is the location were all the shapefiles and anupulaed shapfiles are saved
# under path create five subfolders: cat, riv, hill, cat_step_0, cat_step_1, cat_fixed, hill_fixed
if not os.path.exists(path+'cat'):
    os.mkdir(path+'cat')
if not os.path.exists(path+'riv'):
    os.mkdir(path+'riv')
if not os.path.exists(path+'hill'):
    os.mkdir(path+'hill')
if not os.path.exists(path+'cat_step_0'):
    os.mkdir(path+'cat_step_0')
if not os.path.exists(path+'cat_step_1'):
    os.mkdir(path+'cat_step_1')
if not os.path.exists(path+'cat_fixed'):
    os.mkdir(path+'cat_fixed')
if not os.path.exists(path+'ERA5int'):
    os.mkdir(path+'ERA5int')
if not os.path.exists(path+'hill_fixed'):
    os.mkdir(path+'hill_fixed')
if not os.path.exists(path+'hill_step_0'):
    os.mkdir(path+'hill_step_0')
    

In [ ]:
# downlaod the catchment, river, costal hillslope
for ID in IDs:
    download(path+'cat/',
         http_path+'MERIT_Hydro_v07_Basins_v01_bugfix1/pfaf_level_02/',
        'cat',
        ID)
    download(path+'riv/',
         http_path+'MERIT_Hydro_v07_Basins_v01_bugfix1/pfaf_level_02/',
        'riv',
        ID)
    if ID != '49' # there is no hillslope for 49
        download(path+'hill/',
             http_path+'coastal_hillslopes/',
            'hill',
            ID)